# Homework 1: Dataloader

Try to implement a class named dataloader for the recording demo. It should have the following features:

1. It should load the metadata from the `meta.json` file under the sequence directory.
2. It should return the intrinsics for the given camera_serial.
3. It should return the extrinsics of world space for the given camera_serial.
4. It should return the rgb image for the given camera_serial and frame_id.
5. It should return the depth image for the given camera_serial and frame_id.
6. It should return the points in master camera space for the given camera_serial and frame_id.
7. It should return the points in world space for the given camera_serial and frame_id.

Try to implement the dataloader in the following cell.

In [ ]:
from commons import *


class SequenceLoader:
    def __init__(self, sequence_folder) -> None:
        self.data_folder = Path(sequence_folder).resolve()
        # calibration folder
        self.calib_folder = self.data_folder.parent.parent / "calibration"

        # load metadata
        self.load_metadata()

        # load intrinsics
        self.load_intrinsics()

        # load extrinsics
        self.load_extrinsics()

    def load_metadata(self):
        """ Load metadata from the meta.json file"""

        meta_file = self.data_folder / "meta.json"
        data = read_data_from_json(meta_file)
        self.rs_serials = data["realsense"]["serials"]
        self.rs_width = data["realsense"]["width"]
        self.rs_height = data["realsense"]["height"]
        self.extr_file = (
            self.calib_folder
            / "extrinsics"
            / data["calibration"]["extrinsics"]
            / "extrinsics.json"
        )
        self.subject_id = data["calibration"]["mano"]
        self.mano_file = (
            self.calib_folder / "mano" / self.subject_id  / "mano.json"
        )
        self.object_ids = data["object_ids"]
        self.mano_sides = data["mano_sides"]
        self.num_frames = data["num_frames"]

    def load_intrinsics(self):
        """ Load the list of intrinsics matrices from the calibration folder"""
        self.intrinsics = [
            read_K_matrix_from_json(
                self.calib_folder
                / "intrinsics"
                / f"{s}_{self.rs_width}x{self.rs_height}.json"
            )
            for s in self.rs_serials
        ]

    def load_extrinsics(self):
        """ Load the extrinsics matrices of camera/world space from the calibration folder"""

        extrinsics, rs_master = read_extrinsics_from_json(self.extr_file)
        self.rs_master = rs_master

        tag1 = extrinsics["tag_1"]
        tag1_inv = np.linalg.inv(tag1)

        # the loaded extrinsics are actually from the master to the slave
        self.extrinsics2master = np.stack([extrinsics[s] for s in self.rs_serials])
        self.extrinsics2master_inv = np.stack(
            [np.linalg.inv(t) for t in self.extrinsics2master], axis=0
        )

        # we need to multiply by the tag1_inv to get the slave to world
        self.extrinsics2world = np.stack(
            [tag1_inv @ extrinsics[s] for s in self.rs_serials], axis=0
        )
        self.extrinsics2world_inv = np.stack(
            [np.linalg.inv(t) for t in self.extrinsics2world], axis=0
        )

    def get_rgb_image(self, frame_id, serial):
        """" Get the RGB image for a given frame and camera serial"""
        # write your code here

    def get_depth_image(self, frame_id, serial):
        """ Get the depth image for a given frame and camera serial"""
        # write your code here

    def get_points_camera(self, frame_id, serial):
        """ Get the 3D points in camera space for a given frame and camera serial"""
        # write your code here

    def get_points_world(self, frame_id, serial):
        """ Get the 3D points in world space for a given frame and camera serial"""
        # write your code here

In [ ]:
sequence_folder = PROJ_ROOT / "data/recordings/20231022_193630"
loader = SequenceLoader(sequence_folder)
rs_serials = loader.rs_serials
frame_id = 0

# load the RGBD images for all cameras
rgb_images = [loader.get_rgb_image(frame_id, s) for s in rs_serials]
depth_images = [loader.get_depth_image(frame_id, s) for s in rs_serials]

# display the images
display_images(rgb_images + depth_images, rs_serials+rs_serials)

# prepare the colors for the point cloud
colors = np.vstack(
    [img.reshape(-1, 3).astype(np.float32) / 255.0 for img in rgb_images]
)

# get the points in camera space
points_camera = np.vstack(
    [loader.get_points_camera(frame_id, s) for s in rs_serials]
)
# display the points cloud
pcd_c = o3d.geometry.PointCloud()
pcd_c.points = o3d.utility.Vector3dVector(points_camera)
pcd_c.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw([pcd_c])

# get the points in world space
points_world = np.vstack([loader.get_points_world(frame_id, s) for s in rs_serials])
# display the points cloud
pcd_w = o3d.geometry.PointCloud()
pcd_w.points = o3d.utility.Vector3dVector(points_world)
pcd_w.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw([pcd_w])